In [ ]:
!pip install langchain
!pip install PyPDF2
!pip install qdrant_client
!pip install langchain_openai
!pip install pypdf
!pip install openai

In [2]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import ConversationalRetrievalChain
from qdrant_client import QdrantClient,models
from qdrant_client.http.models import PointStruct
from openai import OpenAI

In [3]:
os.environ["OPENAI_API_KEY"]=""
os.environ["QDRANT_URL"]=""
os.environ["COLLECTION_NAME"]=""

In [4]:
#create new cluseter in qdrant
record=0

connection = QdrantClient(
    url=os.environ.get("QDRANT_URL"),
    api_key="",
)


connection.recreate_collection(
    collection_name=os.environ.get("COLLECTION_NAME"),
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)
print("Create collection reponse:", connection)

info = connection.get_collection(collection_name=os.environ.get("COLLECTION_NAME"))

print("Collection info:", info)
for get_info in info:
  print(get_info)

Create collection reponse: <qdrant_client.qdrant_client.QdrantClient object at 0x798dd76977c0>
Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig

In [5]:
def load_and_split_documents(filepath="baby.pdf"):
  loader = PyPDFLoader(filepath)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=100)
  return text_splitter.split_documents(documents)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
def get_embeddings():
  return OpenAIEmbeddings(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    chunk_size=1
  )

In [7]:
from langchain_openai import ChatOpenAI
def get_chat_model():
  return ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [8]:
def get_document_store(docs, embeddings):
  upsert = Qdrant.from_documents(
    docs,
    embeddings,
    url=os.environ.get("QDRANT_URL"),
    collection_name=os.environ.get("COLLECTION_NAME"),
    force_recreate=True,
    api_key=""
  )
  print(upsert)
  return upsert

In [9]:
def ask_question_with_context(qa, question, chat_history):

    query = ""
    result = qa({"question": question, "chat_history": chat_history})
    print("answer:", result["answer"])
    chat_history = [(query, result["answer"])]
    return chat_history

In [11]:
def main():

    embeddings = get_embeddings()
    docs = load_and_split_documents()
    # print(docs, embeddings)
    doc_store = get_document_store(docs, embeddings)
    llm = get_chat_model()

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=doc_store.as_retriever(),
        return_source_documents=True,
        verbose=False
    )

    chat_history = []
    while True:
        query = input('you: ')
        if query == 'q':
            break
        chat_history = ask_question_with_context(qa, query, chat_history)


if __name__ == "__main__":
    main()


you: 托嬰補助怎麼申請？


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


answer: 要申請托嬰補助，您需要準備以下文件：
1. 未滿二歲兒童托育準公共化服務費用申報表。
2. 申請人及兒童的身分證明文件。
3. 簽訂的托育服務契約書。
4. 申請人其中一方的郵局或金融機構帳戶封面影本。
5. 其他證明文件，如低收入戶、中低收入戶、第二名以上子女等。

您可以在托育事實發生後的15天內，郵寄或親自送交上述文件至托嬰中心或居家保母所屬的居家托育服務中心進行申請。如果對核定結果有疑義，您可以在處分書送達後的30天內提出申復。
you: 誰可以申請托嬰補助？
answer: 未滿2歲兒童托育補助的申請對象包括實際年齡未滿2歲的我國籍兒童，且符合以下情形：
1. 委託人的幼兒完成出生登記或戶籍登記。
2. 送托與政府簽約的公設民營托嬰中心、社區公共托育家園、私立托嬰中心或居家托育人員（保母）。
3. 每週送托時數達30小時以上。
4. 未領取育兒津貼。
5. 未經政府公費安置。
6. 不得指定一對一收托，但有特殊狀況的幼兒除外。
you: q
